In [19]:
import open3d.ml as _ml3d
import open3d as o3d
import open3d.ml.torch as ml3d
import os
import numpy as np

In [20]:
import warnings
warnings.filterwarnings("ignore")

In [21]:
import torch
print(torch.cuda.is_available())

True


In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [24]:
device

device(type='cuda')

In [25]:
cfg_file = r"/mnt/e/PE/Mathew/Codes/Partitions/Open3D-ML/ml3d/configs/kpconv_parislille3d.yml"
cfg = _ml3d.utils.Config.load_from_file(cfg_file)

model = ml3d.models.KPFCNN(**cfg.model)
cfg.dataset['dataset_path'] = r"/mnt/e/PE/Mathew/Codes/Partitions/Open3D-ML/ml3d/datasets/Paris"
dataset = ml3d.datasets.ParisLille3D(cfg.dataset.pop('dataset_path', None), **cfg.dataset)
pipeline = ml3d.pipelines.SemanticSegmentation(model, dataset=dataset, device="gpu", **cfg.pipeline)

In [26]:
ckpt_folder = "./logs/"
os.makedirs(ckpt_folder, exist_ok=True)
ckpt_path = ckpt_folder + "kpconv_parislille3d_202011241550utc.pth"

In [27]:
# load the parameters.
pipeline.load_ckpt(ckpt_path=ckpt_path)

In [28]:
dataset.get_split_list("train")

['/mnt/e/PE/Mathew/Codes/Partitions/Open3D-ML/ml3d/datasets/Paris/training_10_classes/IIITB_Base_train2.ply',
 '/mnt/e/PE/Mathew/Codes/Partitions/Open3D-ML/ml3d/datasets/Paris/training_10_classes/Lille1_1.ply',
 '/mnt/e/PE/Mathew/Codes/Partitions/Open3D-ML/ml3d/datasets/Paris/training_10_classes/Lille1_2.ply',
 '/mnt/e/PE/Mathew/Codes/Partitions/Open3D-ML/ml3d/datasets/Paris/training_10_classes/Paris.ply']

In [29]:
train_split = dataset.get_split("train")
len(train_split)

4

In [30]:
%%time
pipeline.cfg.num_workers = 0
pipeline.cfg.pin_memory = False
pipeline.run_train()

validation: 100%|███████████████████████████████████████████████████████████████████████| 15/15 [00:21<00:00,  1.46s/it]


CPU times: user 1min 29s, sys: 36.3 s, total: 2min 5s
Wall time: 3min 57s


In [32]:
dataset.get_split_list("test")

[]

In [31]:
saved_model_path = './trained_kpconv_model.pth'
torch.save({
    'epoch': result['epoch'],  # The epoch number after training
    'model_state_dict': model.state_dict(),  # Model weights
    'optimizer_state_dict': pipeline.optimizer.state_dict(),  # Optimizer state (useful if you want to resume training)
    'loss': result['loss'],  # Last training loss (optional but useful)
}, saved_model_path)

print(f"Model and training state saved to {saved_model_path}")

TypeError: 'NoneType' object is not subscriptable

In [17]:
"""
TORCH'S -> PTH SAVE

"""

''

In [18]:
pipeline.save_ckpt(ckpt_path="./logs/kpconv_parislille3d_trained.pth")

TypeError: SemanticSegmentation.save_ckpt() got an unexpected keyword argument 'ckpt_path'

In [9]:
test_split = dataset.get_split("test")
test_split_names = dataset.get_split_list("test")
for i in range(len(test_split)):
    data = test_split.get_data(i)
    filename = os.path.splitext(os.path.basename(test_split_names[i]))[0]
    result = pipeline.run_inference(data)
    
    points = data["point"]
    labels = result["predict_labels"]
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(np.zeros_like(points))  # optional placeholder

    output_dir = "Campus Inference"
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, f"{filename}_inference.ply")
    
    with open(output_path, "w") as f:
        f.write("ply\n")
        f.write("format ascii 1.0\n")
        f.write("element vertex {}\n".format(len(points)))
        f.write("property float x\n")
        f.write("property float y\n")
        f.write("property float z\n")
        # f.write("property uchar red\n")
        # f.write("property uchar green\n")
        # f.write("property uchar blue\n")
        f.write("property float scalar_Classification\n")
        # f.write("property float intensity\n")
        f.write("end_header\n")
    
        for p, l in zip(points, labels):
            f.write(f"{p[0]} {p[1]} {p[2]} {float(l)}\n")
    
    print(f"{filename}: Done")

test 0/1: 100%|███████████████████████████████████████████████████████████████| 416877/416877 [03:42<00:00, 1335.33it/s]

Campus_BlockBc_group_10: Done



test 0/1: 100%|███████████████████████████████████████████████████████████████| 416877/416877 [06:54<00:00, 1005.24it/s]

test 0/1: 100%|███████████████████████████████████████████████████████████████| 387053/387053 [02:36<00:00, 1737.90it/s]

Campus_BlockBc_group_11: Done


test 0/1: 100%|███████████████████████████████████████████████████████████████| 209397/209397 [01:19<00:00, 2795.88it/s]

...

Campus_BlockCc_group_7: Done


test 0/1: 100%|████████████████████████████████████████████████████████████████| 846890/846890 [11:52<00:00, 372.52it/s]

Campus_BlockCc_group_8: Done



test 0/1: 100%|████████████████████████████████████████████████████████████████| 846890/846890 [18:30<00:00, 762.36it/s]

test 0/1: 100%|████████████████████████████████████████████████████████████████| 952827/952827 [19:31<00:00, 267.90it/s]

Campus_BlockCc_group_9: Done


In [32]:
# run inference on a single example.
# returns dict with 'predict_labels' and 'predict_scores'.
result = pipeline.run_inference(data)


test 0/1: 100%|███████████████████████████████████████████████████████████████| 443889/443889 [06:06<00:00, 1210.30it/s]

test 0/1: 100%|█████████████████████████████████████████████████████████████| 1571935/1571935 [07:15<00:00, 1375.93it/s]

In [28]:
result.keys()

dict_keys(['predict_labels', 'predict_scores'])

In [27]:
len(result['predict_labels'])

2275320

In [18]:
set(result['predict_labels'])

{0, 1, 2, 4, 5, 6, 7, 8}

In [16]:
import numpy as np
import matplotlib.pyplot as plt

points = data["point"]
labels = result["predict_labels"]

colours = plt.get_cmap("tab20")(labels / labels.max())[:, :3]

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colours)

o3d.io.write_point_cloud("inference_result.ply", pcd)
print("Done")

Done


In [34]:
import numpy as np

points = data["point"]
labels = result["predict_labels"]
# scores = result["predict_scores"]

# vertex = np.array(
#     [tuple(p) + (int(l),) for p, l in zip(points, labels)],
#     dtype=[("x", "f4"), ("y", "f4"), ("z", "f4"), ("label", "i4")]
# )

# ply = PlyData([PlyElement.describe(vertex, "vertex")], text=True)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(np.zeros_like(points))  # optional placeholder

with open("test_result_dijon_9_2.ply", "w") as f:
    f.write("ply\n")
    f.write("format ascii 1.0\n")
    f.write("element vertex {}\n".format(len(points)))
    f.write("property float x\n")
    f.write("property float y\n")
    f.write("property float z\n")
    # f.write("property uchar red\n")
    # f.write("property uchar green\n")
    # f.write("property uchar blue\n")
    f.write("property float scalar_Classification\n")
    # f.write("property float intensity\n")
    f.write("end_header\n")

    for p, l in zip(points, labels):
        f.write(f"{p[0]} {p[1]} {p[2]} {float(l)}\n")

print("Done")

Done


## Raw labelling